In [2]:
!pip install bokeh pathway


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 2

In [3]:
import pandas as pd
import numpy as np
import time

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook

import pathway as pw


In [5]:
# Load CSV
df = pd.read_csv("dataset.csv")

# Map categories to numerical weights
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
traffic_map = {'low': 0.5, 'medium': 1.0, 'high': 1.5}

df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_map)
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df.dropna(subset=['VehicleTypeWeight', 'TrafficLevel'], inplace=True)


In [6]:
# Base price and weights
base_price = 10
weights = {
    'occupancy': 1.0,
    'queue': 0.5,
    'traffic': 0.4,
    'special': 0.3,
    'vehicle': 0.7
}

# Demand calculation
def calculate_demand(row):
    return (
        weights['occupancy'] * row['OccupancyRate'] +
        weights['queue'] * row['QueueLength'] -
        weights['traffic'] * row['TrafficLevel'] +
        weights['special'] * row['IsSpecialDay'] +
        weights['vehicle'] * row['VehicleTypeWeight']
    )

df['Demand'] = df.apply(calculate_demand, axis=1)
min_demand, max_demand = df['Demand'].min(), df['Demand'].max()

# Pricing models
def model1_price(prev_price, occupancy, capacity, alpha=1):
    return prev_price + alpha * (occupancy / capacity)

def model2_price(base_price, demand, min_d, max_d, lambda_=0.5):
    norm_d = (demand - min_d) / (max_d - min_d + 1e-6)
    return np.clip(base_price * (1 + lambda_ * norm_d), 0.5 * base_price, 2 * base_price)


In [7]:
output_notebook()

# Select one parking lot
lot_df = df[df['SystemCodeNumber'] == df['SystemCodeNumber'].unique()[0]].copy().reset_index(drop=True)

# Initialize Bokeh
source = ColumnDataSource(data={'x': [], 'model1': [], 'model2': []})
p = figure(title="Real-Time Price Simulation", x_axis_label="Time Step", y_axis_label="Price ($)")
p.line('x', 'model1', source=source, color='blue', legend_label='Model 1')
p.line('x', 'model2', source=source, color='green', legend_label='Model 2')
p.legend.location = 'top_left'
handle = show(p, notebook_handle=True)

# Stream prices in real time
prev_price = base_price
for i, row in lot_df.iterrows():
    price1 = model1_price(prev_price, row['Occupancy'], row['Capacity'])
    price2 = model2_price(base_price, row['Demand'], min_demand, max_demand)
    prev_price = price1

    source.stream({'x': [i], 'model1': [price1], 'model2': [price2]}, rollover=100)
    push_notebook(handle=handle)
    time.sleep(0.3)


In [16]:
import pathway as pw
import numpy as np

# Schema
class PricingInput(pw.Schema):
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

# UDF for Model 2 pricing
@pw.udf
def model2_price_pathway(Occupancy: int, Capacity: int, QueueLength: int,
                         TrafficConditionNearby: str, IsSpecialDay: int, VehicleType: str) -> float:

    vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
    traffic_map = {'low': 0.5, 'medium': 1.0, 'high': 1.5}

    weights = {
        'occupancy': 1.0,
        'queue': 0.5,
        'traffic': 0.4,
        'special': 0.3,
        'vehicle': 0.7
    }

    base_price = 10
    occ_rate = Occupancy / Capacity
    traffic = traffic_map.get(TrafficConditionNearby, 1.0)
    vehicle = vehicle_map.get(VehicleType, 1.0)

    demand = (
        weights['occupancy'] * occ_rate +
        weights['queue'] * QueueLength -
        weights['traffic'] * traffic +
        weights['special'] * IsSpecialDay +
        weights['vehicle'] * vehicle
    )

    norm_d = (demand - 0) / (6 - 0 + 1e-6)
    price = np.clip(base_price * (1 + 0.5 * norm_d), 0.5 * base_price, 2 * base_price)

    return round(price, 2)

# ✅ Use absolute path
input_table = pw.io.csv.read("/content/dataset.csv", schema=PricingInput)

# Compute price
pricing_output = input_table.select(
    Occupancy=pw.this.Occupancy,
    Capacity=pw.this.Capacity,
    QueueLength=pw.this.QueueLength,
    TrafficConditionNearby=pw.this.TrafficConditionNearby,
    IsSpecialDay=pw.this.IsSpecialDay,
    VehicleType=pw.this.VehicleType,
    Model2Price=model2_price_pathway(
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.QueueLength,
        pw.this.TrafficConditionNearby,
        pw.this.IsSpecialDay,
        pw.this.VehicleType
    )
)

# Optional: Preview few rows
#print(pw.debug.table_to_dataframe(input_table).to_pandas().head())

# Save to file
pw.io.csv.write(pricing_output, "model2_output.csv")

# Run
pw.run()


Output()

KeyboardInterrupt: 

In [17]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
import pandas as pd
import numpy as np

output_notebook()

# Read both dataset and model2 output
df = pd.read_csv("dataset.csv")
model2_df = pd.read_csv("model2_output.csv")

# Merge (assuming same order)
df = df.iloc[:len(model2_df)].copy()
df['Model2Price'] = model2_df['Model2Price']

# Model 1 calculation
base_price = 10
model1_prices = []
for i, row in df.iterrows():
    price = base_price + 1.0 * (row['Occupancy'] / row['Capacity'])  # α = 1
    model1_prices.append(price)
    base_price = price

df['Model1Price'] = model1_prices

# Plot using Bokeh
source = ColumnDataSource(data={
    'x': list(range(len(df))),
    'model1': df['Model1Price'],
    'model2': df['Model2Price']
})

p = figure(title="Model 1 vs Model 2 Pricing", x_axis_label="Time Step", y_axis_label="Price ($)", width=800, height=400)
p.line('x', 'model1', source=source, color='blue', legend_label='Model 1', line_width=2)
p.line('x', 'model2', source=source, color='green', legend_label='Model 2', line_width=2)
p.legend.location = 'top_left'
show(p)


In [21]:
readme_content = """
#  Dynamic Pricing for Urban Parking Lots

Capstone Project – Summer Analytics 2025
Hosted by Consulting & Analytics Club × Pathway

##  Project Overview

This project implements real-time dynamic pricing for urban parking lots using data such as occupancy, traffic, queue length, and vehicle type.

We implemented:
- Model 1: Linear pricing based on occupancy
-  Model 2: Demand-based pricing
-  Model 3: Not implemented (as per scope)

## Tech Stack

- Python
- NumPy & Pandas
- Bokeh (Real-time visualization)
- Pathway (Real-time data streaming engine)
- Google Colab

## Pricing Models

**Model 1**:
P(t+1) = P(t) + α × (Occupancy / Capacity)

**Model 2**:
Demand = α·OccRate + β·Queue − γ·Traffic + δ·SpecialDay + ε·VehicleType
Price = BasePrice × (1 + λ × NormalizedDemand)

## Architecture Flow

1. Input dataset from 14 parking lots
2. Preprocessing vehicle & traffic weights
3. Apply Model 1 & 2 logic
4. Run pricing in real-time using Pathway
5. Display graph using Bokeh

## Architecture Diagram

![System Architecture](architecture.png)

## Files Included

- `notebook.ipynb`: Colab notebook
- `dataset.csv`: Input dataset
- `model2_output.csv`: Pathway output
- `README.md`: This file
- `architecture.png`: System diagram

##  Run Instructions

1. Upload all files to Colab
2. Run all cells in order
3. Check final graph and `model2_output.csv` in sidebar
"""

with open("/content/README.md", "w") as f:
    f.write(readme_content)

print("README.md created at /content/README.md")


README.md created at /content/README.md


In [19]:
import os

# List all files in current directory
print(os.listdir("/content"))


['.config', 'dataset.csv', 'model2_output.csv', '.ipynb_checkpoints', 'sample_data']
